In [8]:
!pip install pinecone pinecone-text pinecone-notebooks

                                              0.0/421.9 kB ? eta -:--:--
     -------------                          153.6/421.9 kB 4.6 MB/s eta 0:00:01
     ----------------------------------     378.9/421.9 kB 4.7 MB/s eta 0:00:01
     -------------------------------------- 421.9/421.9 kB 3.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: C:\Users\shiva\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
api_key="pcsk_4dmNH8_D5o8QwUqLRWhSZPiT7DTep3mKAEm3TinSxFHS9HqkkKbK6Qf2TJXZRsh1Z7necT"

In [11]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [12]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
from pinecone import Pinecone,ServerlessSpec

index_name="hybrid-search-langchain-pinecone"
pc=Pinecone(api_key=api_key)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, #dimensions of dense model
        metric="dotproduct",
        spec=ServerlessSpec(cloud="aws",region="us-east-1")
  )


In [13]:
index=pc.Index(index_name)
index

In [14]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_community.embeddings import HuggingFaceEmbeddings


In [15]:
import sympy

In [16]:
Name: sympy
Version: 1.12

In [17]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

C:\Users\shiva\AppData\Local\Temp\ipykernel_14528\2855845696.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [ ]:
!pip install huggingface_embeddings

In [18]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, multi_process=False, show_progress=False)

In [19]:
from pinecone_text.sparse import BM25Encoder

In [20]:
bm25_encoder=BM25Encoder().default()

In [21]:
sentence=["In 2023 I visited paris","In 2022, I visited New york","In 2021,I visited New Orelans"]

In [22]:
bm25_encoder.fit(sentence)

  0%|          | 0/3 [00:00<?, ?it/s]

In [23]:
bm25_encoder.dump("bm25_values.json")

In [24]:
bm25_encoder=BM25Encoder().load("bm25_values.json")

In [25]:
bm25_encoder

In [26]:
bm25_encoder

In [27]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [28]:
retriever.add_texts(
    [
        "In 2023 I visited paris",
        "In 2022 I visited New York",
        "In 2021 I visited New Orelans",
    ]
)

  0%|          | 0/1 [00:00<?, ?it/s]

In [30]:
retriever.invoke("which city did i visit in 2021, give that only")

[Document(metadata={'score': 0.355292916}, page_content='In 2021 I visited New Orelans'),
 Document(metadata={'score': 0.323967189}, page_content='In 2022 I visited New York'),
 Document(metadata={'score': 0.300015748}, page_content='In 2023 I visited paris')]